# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,imbituba,-28.2400,-48.6703,23.28,94,100,4.68,BR,1697574269
1,1,verkhniye sergi,56.6469,59.5556,8.69,66,98,6.03,RU,1697574269
2,2,waitangi,-43.9535,-176.5597,11.57,76,0,1.79,NZ,1697574270
3,3,adamstown,-25.0660,-130.1015,22.15,87,100,4.16,PN,1697574270
4,4,puerto natales,-51.7236,-72.4875,12.25,54,75,9.26,CL,1697574270


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", size = "Humidity", frame_width = 700, frame_height = 500, color = "City", scale = 1, alpha = 0.5, xlabel = "Longitude", ylabel = "Latitude")

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] >= 15) & (city_data_df["Max Temp"] <= 26) & 
                                    (city_data_df["Humidity"] >= 20) & (city_data_df["Humidity"] <= 60) & 
                                    (city_data_df["Wind Speed"] >= 4) & (city_data_df["Wind Speed"] <= 24)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,swift current,50.2834,-107.8014,15.45,58,20,12.86,CA,1697574061
38,38,ovalle,-30.5983,-71.2003,24.10,30,100,4.52,CL,1697574278
172,172,concordia,-31.3930,-58.0209,24.32,33,99,4.02,AR,1697574054
207,207,billings,45.7833,-108.5007,22.94,31,75,12.86,US,1697574300
229,229,princeton,25.5384,-80.4089,25.57,50,0,4.63,US,1697574302
298,298,pratt,37.6439,-98.7376,25.42,25,0,7.72,US,1697574262
342,342,comodoro rivadavia,-45.8667,-67.5000,16.92,34,0,5.66,AR,1697574348
353,353,cafayate,-26.0730,-65.9761,20.47,58,59,5.20,AR,1697574351
372,372,rawson,-43.3002,-65.1023,15.92,42,3,6.41,AR,1697574355
385,385,mount isa,-20.7333,139.5000,21.87,20,0,4.12,AU,1697574359


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
17,swift current,CA,50.2834,-107.8014,58,
38,ovalle,CL,-30.5983,-71.2003,30,
172,concordia,AR,-31.3930,-58.0209,33,
207,billings,US,45.7833,-108.5007,31,
229,princeton,US,25.5384,-80.4089,50,
298,pratt,US,37.6439,-98.7376,25,
342,comodoro rivadavia,AR,-45.8667,-67.5000,34,
353,cafayate,AR,-26.0730,-65.9761,58,
372,rawson,AR,-43.3002,-65.1023,42,
385,mount isa,AU,-20.7333,139.5000,20,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": 'accommodation.hotel', "limit": 1, "apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude=row["Lat"]
    longitude=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
swift current - nearest hotel: Days Inn
ovalle - nearest hotel: Hotel Roxy
concordia - nearest hotel: D'Charruas
billings - nearest hotel: Best Western Plus Clocktower Inn
princeton - nearest hotel: La Quinta Inn & Suites by Wyndham Miami Cutler Bay
pratt - nearest hotel: Best Western Plus
comodoro rivadavia - nearest hotel: No hotel found
cafayate - nearest hotel: Il Cavallino
rawson - nearest hotel: Hotel Deportivo
mount isa - nearest hotel: Ibis Styles
carnarvon - nearest hotel: No hotel found
rio gallegos - nearest hotel: Amérian Río Gallegos Apart & Suites
miles city - nearest hotel: Historic Olive Hotel
cedeira - nearest hotel: Hospedería Casa Cordobelas
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
17,swift current,CA,50.2834,-107.8014,58,Days Inn
38,ovalle,CL,-30.5983,-71.2003,30,Hotel Roxy
172,concordia,AR,-31.3930,-58.0209,33,D'Charruas
207,billings,US,45.7833,-108.5007,31,Best Western Plus Clocktower Inn
229,princeton,US,25.5384,-80.4089,50,La Quinta Inn & Suites by Wyndham Miami Cutler...
298,pratt,US,37.6439,-98.7376,25,Best Western Plus
342,comodoro rivadavia,AR,-45.8667,-67.5000,34,No hotel found
353,cafayate,AR,-26.0730,-65.9761,58,Il Cavallino
372,rawson,AR,-43.3002,-65.1023,42,Hotel Deportivo
385,mount isa,AU,-20.7333,139.5000,20,Ibis Styles


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", size = "Humidity", frame_width = 700, frame_height = 500, color = "City", scale = 1.5, alpha = 0.5, xlabel = "Longitude", ylabel = "Latitude", hover_cols = ["Hotel Name", "Country"])

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)